In [5]:
#KGAT_746a8c48819a19cbaf8ca0048244831b

In [4]:
!pip install opendatasets
!pip install pandas

Библиотеки

In [9]:
import opendatasets as od
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


Скачиваем датасет  
Чтение датасета  
Приводим таргет к числовому типу, удаляя строки, которые не удалось конвертировать  
В House price of unit area есть цена 0, что может помешать модели, поэтому удаляем  
Формируем признаки и целевую переменную Таргет - цена Признаки использую все, кроме цены и времени, осталяя только числовые характеристики  
Делюсь на обучающую и тестовую выборки

In [7]:
od.download("https://www.kaggle.com/datasets/kirbysasuke/house-price-prediction-simplified-for-regression")

pd.set_option("display.max_columns", None)
df = pd.read_csv("house-price-prediction-simplified-for-regression/Real_Estate.csv")

df["House price of unit area"] = pd.to_numeric(df["House price of unit area"], errors="coerce")
df = df.dropna(subset=["House price of unit area"])

df = df[df["House price of unit area"] > 0].copy()

y = df["House price of unit area"]
X = df.drop(columns=["House price of unit area", "Transaction date"])

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: angrytea
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/kirbysasuke/house-price-prediction-simplified-for-regression


100%|██████████| 17.4k/17.4k [00:00<00:00, 16.4MB/s]

Строю базовую модель дерева решений для регрессии с настройками по умолчанию

Базовое дерево решений без подбора гиперпараметров даёт очень плохое качество: MAE ≈ 15.8, RMSE ≈ 18.6 и сильно отрицательный R^2 (≈ -0.91). Это значит, что модель на тесте работает даже хуже, чем тривиальный предиктор «всегда средняя цена». Такое дерево явно переобучается на небольшой выборке и практически бесполезно как регрессионная модель

In [10]:
tree_base = DecisionTreeRegressor(random_state=42)

tree_base.fit(X_train, y_train)
y_pred_base = tree_base.predict(X_test)

mae_base = mean_absolute_error(y_test, y_pred_base)
rmse_base = np.sqrt(mean_squared_error(y_test, y_pred_base))
r2_base = r2_score(y_test, y_pred_base)

print("Бейзлайн")
print("MAE:", mae_base)
print("RMSE:", rmse_base)
print("R^2:", r2_base)

Бейзлайн
MAE: 15.793130604968212
RMSE: 18.57892447102105
R^2: -0.9084360232822504


Гипотеза 1 - аккуратная регуляризация дерева (ограничение глубины и минимального числа объектов в узлах) поможет снизить переобучение и улучшить MAE на тесте. С помощью GridSearchCV подбираю max_depth, min_samples_split и min_samples_leaf, оптимизируя MAE

После подбора глубины и минимального числа объектов в узле/листе качество заметно улучшается относительно бейзлайна: MAE падает с ~15.8 до ~12.1, RMSE тоже заметно уменьшается. R^2 вырастает с -0.91 до примерно -0.15 - всё ещё хуже константной модели, но уже не провал. Гипотеза частично подтверждается: жёсткий контроль структуры дерева действительно снижает переобучение и делает предсказания разумнее, но одного только тюнинга max_depth/min_samples_* всё равно недостаточно, чтобы получить действительно хорошую регрессию на этих данных

In [11]:
param_grid_tree = {
    "model__max_depth": [None, 3, 5, 7, 9, 12],
    "model__min_samples_split": [2, 5, 10, 20],
    "model__min_samples_leaf": [1, 2, 4, 8],
}

tree_pipe = Pipeline(steps=[
    ("model", DecisionTreeRegressor(random_state=42))
])

tree_gs = GridSearchCV(
    estimator=tree_pipe,
    param_grid=param_grid_tree,
    scoring="neg_mean_absolute_error",
    cv=3,
    n_jobs=-1
)

tree_gs.fit(X_train, y_train)

print("Лучшие параметры:", tree_gs.best_params_)
print("Лучший MAE на CV:", -tree_gs.best_score_)

best_tree_1 = tree_gs.best_estimator_
y_pred_1 = best_tree_1.predict(X_test)

mae_1 = mean_absolute_error(y_test, y_pred_1)
rmse_1 = np.sqrt(mean_squared_error(y_test, y_pred_1))
r2_1 = r2_score(y_test, y_pred_1)

print("\nГипотеза 1")
print("MAE:", mae_1)
print("RMSE:", rmse_1)
print("R^2:", r2_1)

Лучшие параметры: {'model__max_depth': 7, 'model__min_samples_leaf': 4, 'model__min_samples_split': 20}
Лучший MAE на CV: 10.26159260635103

Гипотеза 1
MAE: 12.096726754385621
RMSE: 14.397518280072877
R^2: -0.14607158350214489


Гипотеза 2 - добавляю новый столбец log_dist_MRT, использую его вместо исходного расстояния и снова провожу подбор гиперпараметров дерева

Лучшие параметры и итоговые метрики полностью совпали с гипотезой 1: MAE ≈ 12.1, RMSE ≈ 14.4, R^2 ≈ -0.15. Это говорит о том, что в текущей конфигурации дерево либо почти не использует признак расстояния в качестве разбиения, либо его линейная/логарифмическая форма принципиально не влияет на структуру дерева. Гипотеза про log(distance) по сути не приносит дополнительного выигрыша по сравнению с уже настроенной моделью

In [12]:
df_log = df.copy()
df_log["log_dist_MRT"] = np.log1p(df_log["Distance to the nearest MRT station"])

features_log = [
    "House age",
    "log_dist_MRT",
    "Number of convenience stores",
    "Latitude",
    "Longitude",
]

X_log = df_log[features_log]
y_log = df_log["House price of unit area"]

X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(
    X_log, y_log,
    test_size=0.2,
    random_state=42
)

tree_log_pipe = Pipeline(steps=[
    ("model", DecisionTreeRegressor(random_state=42))
])

param_grid_tree_log = param_grid_tree

tree_log_gs = GridSearchCV(
    estimator=tree_log_pipe,
    param_grid=param_grid_tree_log,
    scoring="neg_mean_absolute_error",
    cv=3,
    n_jobs=-1
)

tree_log_gs.fit(X_train_log, y_train_log)

print("Лучшие параметры:", tree_log_gs.best_params_)
print("Лучший MAE на CV:", -tree_log_gs.best_score_)

best_tree_2 = tree_log_gs.best_estimator_
y_pred_2 = best_tree_2.predict(X_test_log)

mae_2 = mean_absolute_error(y_test_log, y_pred_2)
rmse_2 = np.sqrt(mean_squared_error(y_test_log, y_pred_2))
r2_2 = r2_score(y_test_log, y_pred_2)

print("\nDecisionTreeRegressor + log(расстояния до MRT)")
print("MAE:", mae_2)
print("RMSE:", rmse_2)
print("R^2:", r2_2)


Лучшие параметры: {'model__max_depth': 7, 'model__min_samples_leaf': 4, 'model__min_samples_split': 20}
Лучший MAE на CV: 10.26159260635103

DecisionTreeRegressor + log(расстояния до MRT)
MAE: 12.096726754385621
RMSE: 14.397518280072877
R^2: -0.14607158350214489


Гипотеза 3 - цены могут иметь тяжёлый хвост (очень дорогие квартиры), поэтому логарифмирование таргета снижает влияние экстремально дорогих объектов

Логарифмирование целевой переменной даёт уже качественно другой результат: MAE снижается примерно до 9.68, RMSE — до ~11.49, а R^2 становится положительным (около 0.27). Это лучший результат среди всех вариантов дерева решений в этой лабораторной, и по уровню качества он уже сравним с линейными моделями и KNN. Дерево в лог-шкале лучше справляется с разбросом цен и меньше «ломается» на дорогих объектах. Гипотеза подтверждается: переход к логарифму цены делает задачу более удобной для дерева решений и заметно улучшает его обобщающую способность

In [13]:
y_log_target = np.log1p(df["House price of unit area"])
X_t = df.drop(columns=["House price of unit area", "Transaction date"])

X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(
    X_t, y_log_target,
    test_size=0.2,
    random_state=42
)

tree_logy_pipe = Pipeline(steps=[
    ("model", DecisionTreeRegressor(random_state=42))
])

param_grid_tree_logy = param_grid_tree

tree_logy_gs = GridSearchCV(
    estimator=tree_logy_pipe,
    param_grid=param_grid_tree_logy,
    scoring="neg_mean_absolute_error",
    cv=3,
    n_jobs=-1
)

tree_logy_gs.fit(X_train_t, y_train_t)

print("Лучшие параметры:", tree_logy_gs.best_params_)
print("Лучший MAE на CV:", -tree_logy_gs.best_score_)

best_tree_3 = tree_logy_gs.best_estimator_

y_pred_log_t = best_tree_3.predict(X_test_t)
y_pred_t = np.expm1(y_pred_log_t)

y_test_true = np.expm1(y_test_t)

mae_3 = mean_absolute_error(y_test_true, y_pred_t)
rmse_3 = np.sqrt(mean_squared_error(y_test_true, y_pred_t))
r2_3 = r2_score(y_test_true, y_pred_t)

print("\nDecisionTreeRegressor + логарифм таргета")
print("MAE:", mae_3)
print("RMSE:", rmse_3)
print("R^2:", r2_3)


Лучшие параметры: {'model__max_depth': 3, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2}
Лучший MAE на CV: 0.38411858154593076

DecisionTreeRegressor + логарифм таргета
MAE: 9.683404490734594
RMSE: 11.487476850632104
R^2: 0.2703984202546508


Собственная реализация  
Модель устроена по схеме CART: в методе fit рекурсивно строю дерево, на каждом узле перебираю признаки и пороги разбиения, считаю уменьшение среднеквадратичной ошибки (MSE) и выбираю лучший сплит. Рост дерева останавливается по условиям max_depth, min_samples_split, min_samples_leaf или если дальнейшее разбиение не даёт выигрыша по метрике. В листьях хранится среднее значение таргета по объектам, попавшим в лист. В методе predict для каждого объекта последовательно спускаюсь от корня по условиям feature <= threshold до листа и возвращаю сохранённое среднее. Параметры работают аналогично DecisionTreeRegressor из sklearn, поэтому модель можно использовать внутри Pipeline и настраивать через GridSearchCV

In [14]:
class MyDecisionTreeRegressor(BaseEstimator, RegressorMixin):
    def __init__(self,
                 max_depth=None,
                 min_samples_split=2,
                 min_samples_leaf=1,
                 random_state=None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.random_state = random_state

    class Node:
        def __init__(self, is_leaf, prediction=None,
                     feature_index=None, threshold=None,
                     left=None, right=None):
            self.is_leaf = is_leaf
            self.prediction = prediction
            self.feature_index = feature_index
            self.threshold = threshold
            self.left = left
            self.right = right

    def _mse(self, y):
        if y.size == 0:
            return 0.0
        mean = np.mean(y)
        return np.mean((y - mean) ** 2)

    def _best_split(self, X, y):
        n_samples, n_features = X.shape
        if n_samples < self.min_samples_split:
            return None, None

        best_feature = None
        best_threshold = None
        best_mse = self._mse(y)
        base_mse = best_mse

        for feature in range(n_features):
            x_col = X[:, feature]
            sort_idx = np.argsort(x_col)
            x_sorted = x_col[sort_idx]
            y_sorted = y[sort_idx]

            unique_vals = np.unique(x_sorted)
            if unique_vals.size <= 1:
                continue

            thresholds = (unique_vals[:-1] + unique_vals[1:]) / 2.0

            for thr in thresholds:
                left_mask = x_sorted <= thr
                right_mask = ~left_mask

                y_left = y_sorted[left_mask]
                y_right = y_sorted[right_mask]

                if y_left.size < self.min_samples_leaf or y_right.size < self.min_samples_leaf:
                    continue

                mse_left = self._mse(y_left)
                mse_right = self._mse(y_right)
                mse_split = (y_left.size * mse_left + y_right.size * mse_right) / n_samples

                if mse_split < best_mse:
                    best_mse = mse_split
                    best_feature = feature
                    best_threshold = thr

        if best_feature is None:
            return None, None

        if best_mse >= base_mse:
            return None, None

        return best_feature, best_threshold

    def _build_tree(self, X, y, depth):
        n_samples = y.size
        prediction = np.mean(y)

        if n_samples < self.min_samples_split:
            return self.Node(is_leaf=True, prediction=prediction)

        if self.max_depth is not None and depth >= self.max_depth:
            return self.Node(is_leaf=True, prediction=prediction)

        feature, threshold = self._best_split(X, y)
        if feature is None:
            return self.Node(is_leaf=True, prediction=prediction)

        left_mask = X[:, feature] <= threshold
        right_mask = ~left_mask

        X_left, y_left = X[left_mask], y[left_mask]
        X_right, y_right = X[right_mask], y[right_mask]

        left_child = self._build_tree(X_left, y_left, depth + 1)
        right_child = self._build_tree(X_right, y_right, depth + 1)

        return self.Node(
            is_leaf=False,
            prediction=None,
            feature_index=feature,
            threshold=threshold,
            left=left_child,
            right=right_child
        )

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)
        self.root_ = self._build_tree(X, y, depth=0)
        return self

    def _predict_row(self, x, node):
        if node.is_leaf:
            return node.prediction
        if x[node.feature_index] <= node.threshold:
            return self._predict_row(x, node.left)
        else:
            return self._predict_row(x, node.right)

    def predict(self, X):
        X = np.asarray(X)
        preds = np.array([self._predict_row(row, self.root_) for row in X])
        return preds

Бейзлайн собственной реализации

Собственное дерево решений в базовой конфигурации даёт почти те же метрики, что и библиотечное: MAE ≈ 15.8, RMSE ≈ 18.6, R^2 ≈ -0.90. То есть на тесте модель всё ещё сильно хуже константного предиктора по средней цене. Это значит, что без ограничения глубины и размеров узлов дерево жёстко переобучается на небольшой выборке

In [15]:
my_tree_base = MyDecisionTreeRegressor(
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1
)

my_tree_base.fit(X_train.values, y_train.values)
y_pred_my_base = my_tree_base.predict(X_test.values)

mae_my_base = mean_absolute_error(y_test, y_pred_my_base)
rmse_my_base = np.sqrt(mean_squared_error(y_test, y_pred_my_base))
r2_my_base = r2_score(y_test, y_pred_my_base)

print("Бейзлайн")
print("MAE:", mae_my_base)
print("RMSE:", rmse_my_base)
print("R^2:", r2_my_base)


Бейзлайн
MAE: 15.776151832857598
RMSE: 18.55395950475414
R^2: -0.9033106425561108


Гипотеза 1

После подбора max_depth, min_samples_split и min_samples_leaf качество становится существенно лучше: MAE падает примерно до 12.1, RMSE до ~14.4, а R² поднимается с ~ -0.90 до ~ -0.15. Как и в случае со sklearn-деревом, модель всё ещё формально хуже константной по R^2, но уже не катастрофически. Гипотеза подтверждается: аккуратный контроль структуры дерева заметно снижает переобучение и делает предсказания более адекватными, хотя до действительно хорошего качества всё ещё далеко

In [19]:
my_tree_pipe = Pipeline(steps=[
    ("model", MyDecisionTreeRegressor())
])

param_grid_my_tree_1 = {
    "model__max_depth": [None, 3, 5, 7, 9, 12],
    "model__min_samples_split": [2, 5, 10, 20],
    "model__min_samples_leaf": [1, 2, 4, 8],
}

my_tree_gs_1 = GridSearchCV(
    estimator=my_tree_pipe,
    param_grid=param_grid_my_tree_1,
    scoring="neg_mean_absolute_error",
    cv=3,
    n_jobs=-1
)

my_tree_gs_1.fit(X_train, y_train)

print("Лучший параметры:", my_tree_gs_1.best_params_)
print("Лучший MAE на CV:", -my_tree_gs_1.best_score_)

best_my_tree_1 = my_tree_gs_1.best_estimator_
y_pred_my_1 = best_my_tree_1.predict(X_test)

mae_my_1 = mean_absolute_error(y_test, y_pred_my_1)
rmse_my_1 = np.sqrt(mean_squared_error(y_test, y_pred_my_1))
r2_my_1 = r2_score(y_test, y_pred_my_1)

print("\nГипотеза 1")
print("MAE:", mae_my_1)
print("RMSE:", rmse_my_1)
print("R^2:", r2_my_1)


Лучший параметры: {'model__max_depth': 7, 'model__min_samples_leaf': 4, 'model__min_samples_split': 20}
Лучший MAE на CV: 10.26159260635103

Гипотеза 1
MAE: 12.096726754385621
RMSE: 14.397518280072875
R^2: -0.14607158350214466


Гипотеза 2

Добавление логарифма расстояния до MRT к собственной реализации приводит к идентичным лучшим гиперпараметрам и тем же метрикам, что и в гипотезе 1: MAE ≈ 12.1, RMSE ≈ 14.4, R^2 ≈ -0.15. По сути дерево либо почти не использует признак расстояния как ключевой для разбиений, либо форма расстояния (линейная vs log1p) мало что меняет в структуре. Гипотеза про log(distance) для дерева решений в этой задаче эффекта не даёт — качество осталось на уровне «только тюнинг структуры»

In [20]:
df_log2 = df.copy()
df_log2["log_dist_MRT"] = np.log1p(df_log2["Distance to the nearest MRT station"])

features_log2 = [
    "House age",
    "log_dist_MRT",
    "Number of convenience stores",
    "Latitude",
    "Longitude",
]

X_log2 = df_log2[features_log2]
y_log2 = df_log2["House price of unit area"]

X_train_log2, X_test_log2, y_train_log2, y_test_log2 = train_test_split(
    X_log2, y_log2,
    test_size=0.2,
    random_state=42
)

my_tree_pipe_2 = Pipeline(steps=[
    ("model", MyDecisionTreeRegressor())
])

param_grid_my_tree_2 = param_grid_my_tree_1

my_tree_gs_2 = GridSearchCV(
    estimator=my_tree_pipe_2,
    param_grid=param_grid_my_tree_2,
    scoring="neg_mean_absolute_error",
    cv=3,
    n_jobs=-1
)

my_tree_gs_2.fit(X_train_log2, y_train_log2)

print("Лучшие параметры:", my_tree_gs_2.best_params_)
print("Лучший MAE на CV:", -my_tree_gs_2.best_score_)

best_my_tree_2 = my_tree_gs_2.best_estimator_
y_pred_my_2 = best_my_tree_2.predict(X_test_log2)

mae_my_2 = mean_absolute_error(y_test_log2, y_pred_my_2)
rmse_my_2 = np.sqrt(mean_squared_error(y_test_log2, y_pred_my_2))
r2_my_2 = r2_score(y_test_log2, y_pred_my_2)

print("\nMyDecisionTreeRegressor + log(расстояния до MRT)")
print("MAE:", mae_my_2)
print("RMSE:", rmse_my_2)
print("R^2:", r2_my_2)


Лучшие параметры: {'model__max_depth': 7, 'model__min_samples_leaf': 4, 'model__min_samples_split': 20}
Лучший MAE на CV: 10.26159260635103

MyDecisionTreeRegressor + log(расстояния до MRT)
MAE: 12.096726754385621
RMSE: 14.397518280072875
R^2: -0.14607158350214466


Гипотеза 3

При переходе к логарифму целевой переменной качество резко вырастает: MAE ≈ 9.65, RMSE ≈ 11.48, R^2 ≈ 0.27 - фактически те же значения, что и у библиотечного DecisionTreeRegressor с той же идеей. Модель наконец-то становится лучше константного предиктора и уже приближается по качеству к результатам линейных моделей и KNN. Гипотеза полностью подтверждается: логарифмирование цены сглаживает влияние дорогих объектов и делает задачу более удобной для кусочно-постоянных предсказаний дерева

In [21]:
y_logtarget2 = np.log1p(df["House price of unit area"])
X_t2 = df.drop(columns=["House price of unit area", "Transaction date"])

X_train_t2, X_test_t2, y_train_t2, y_test_t2 = train_test_split(
    X_t2, y_logtarget2,
    test_size=0.2,
    random_state=42
)

my_tree_pipe_3 = Pipeline(steps=[
    ("model", MyDecisionTreeRegressor())
])

param_grid_my_tree_3 = param_grid_my_tree_1

my_tree_gs_3 = GridSearchCV(
    estimator=my_tree_pipe_3,
    param_grid=param_grid_my_tree_3,
    scoring="neg_mean_absolute_error",
    cv=3,
    n_jobs=-1
)

my_tree_gs_3.fit(X_train_t2, y_train_t2)

print("Лучшие параметры:", my_tree_gs_3.best_params_)
print("Лучший MAE на CV:", -my_tree_gs_3.best_score_)

best_my_tree_3 = my_tree_gs_3.best_estimator_

y_pred_log_t2 = best_my_tree_3.predict(X_test_t2)
y_pred_t2 = np.expm1(y_pred_log_t2)

y_test_true2 = np.expm1(y_test_t2)

mae_my_3 = mean_absolute_error(y_test_true2, y_pred_t2)
rmse_my_3 = np.sqrt(mean_squared_error(y_test_true2, y_pred_t2))
r2_my_3 = r2_score(y_test_true2, y_pred_t2)

print("\nMyDecisionTreeRegressor + логарифм таргета")
print("MAE:", mae_my_3)
print("RMSE:", rmse_my_3)
print("R^2:", r2_my_3)


Лучшие параметры: {'model__max_depth': 3, 'model__min_samples_leaf': 1, 'model__min_samples_split': 5}
Лучший MAE на CV: 0.38527464244999027

MyDecisionTreeRegressor + логарифм таргета
MAE: 9.653658712273353
RMSE: 11.483049920745986
R^2: 0.2709606451414325


В регрессионной задаче по ценам недвижимости дерево решений в «сыром» виде даёт провальные результаты, а основное улучшение приходит от двух шагов тюнинга структуры (max_depth, min_samples_split, min_samples_leaf) и перехода к логарифму таргета. Лог(distance) в признаках практически ничего не меняет. Собственная реализация MyDecisionTreeRegressor по всем сценариям воспроизводит поведение sklearn-дерева: метрики почти совпадают, те же гиперпараметры оказываются оптимальными, а лучший результат даёт именно логарифмирование цены. Это позволяет считать реализацию корректной, а главный вывод — как и в прошлых лабораторных, качество в итоге определяется не столько ручным кодом модели, сколько грамотной трансформацией целевой переменной и регуляризацией структуры дерева